In [1]:
import copy
import numpy as np
import os
import time
import palettable
import h5py

In [2]:
import yt
import trident
import trident.line_database as line_database
import unyt

In [3]:
import matplotlib.colors as plt_colors
import matplotlib.gridspec as gridspec
from matplotlib.transforms import blended_transform_factory
import matplotlib.patheffects as path_effects

In [4]:
import firestudio.studios.gas_studio as gas_studio
from abg_python.snapshot_utils import openSnapshot
from abg_python.cosmoExtractor import extractDiskFromSnapdict,offsetRotateSnapshot

don't have phil's colormaps


In [5]:
import stained_glass.utils.file_management as file_management

In [6]:
import verdict
import galaxy_dive.analyze_data.halo_data as halo_data
import galaxy_dive.plot_data.plotting as plotting

# Setup

In [7]:
sim_name = 'm12i'
snum = 600
resolution = 7100
physics = 'core'
subsubdir = None
mt_halo_id = 0
mt_kwargs = {
    'tag': 'smooth',
}

In [8]:
sim_kwargs = dict(
    sim_name = sim_name,
    resolution = resolution,
    physics = physics,
    subsubdir = subsubdir,
)

In [9]:
line_list = ['H', 'C', 'N', 'O', 'Mg', 'Si']
sidelength = 600. # in kpc
sightline_len = 1e3 # in kpc
species_list = [
#     'H I',
#     'Mg II',
    'Si II', 
#     'Si III',
#     'Si IV',
#     'C II',
#     'C III',
    'C IV',
    'O VI',
#     'O VIII',
]
use_stored_projections = False

In [10]:
fm = file_management.FileManager( 'mock_obs' )

### YT Data

In [11]:
time_start = time.time()

ds = yt.load( fm.get_yt_filepath(
    sim_name,
    snum,
    resolution=resolution,
    physics=physics,
    subsubdir=subsubdir
) )

time_end = time.time()

print( 'Loading took {:.3g} seconds'.format( time_end - time_start ) )

yt : [INFO     ] 2020-09-16 19:42:34,498 Calculating time from 1.000e+00 to be 4.355e+17 seconds
yt : [INFO     ] 2020-09-16 19:42:34,499 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2020-09-16 19:42:34,527 Parameters: current_time              = 4.3545571088051405e+17 s
yt : [INFO     ] 2020-09-16 19:42:34,528 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2020-09-16 19:42:34,528 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-09-16 19:42:34,529 Parameters: domain_right_edge         = [60000. 60000. 60000.]
yt : [INFO     ] 2020-09-16 19:42:34,529 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2020-09-16 19:42:34,530 Parameters: current_redshift          = 0.0
yt : [INFO     ] 2020-09-16 19:42:34,530 Parameters: omega_lambda              = 0.728
yt : [INFO     ] 2020-09-16 19:42:34,530 Parameters: omega_matter              = 0.272
yt : [INFO     ] 2020-09-16 19:42:34,531 Parameters: omega_radiation           = 0.

Loading took 0.348 seconds


### Halo Data

In [12]:
hd = halo_data.HaloData(
    data_dir = fm.get_halo_dir( **sim_kwargs ),
    mt_kwargs = mt_kwargs,
)

### Alex Data

In [13]:
sim_dir = fm.get_sim_dir( **sim_kwargs )
gas_snap = openSnapshot( sim_dir, snum, 0 )
star_snap = openSnapshot( sim_dir, snum, 4 )

This is a cosmological snapshot... converting to physical units
This is a cosmological snapshot... converting to physical units


## Add Ion Fields

In [14]:
trident.add_ion_fields(ds, ions=species_list, ftype="PartType0")

yt : [INFO     ] 2020-09-16 19:43:54,125 Allocating for 1.470e+08 particles
Loading particle index: 100%|██████████| 270/270 [00:00<00:00, 3586.90it/s]


In [15]:
# Get the line data filepath
trident_dir = os.path.dirname( trident.__file__ )
linedb_fp = os.path.join( trident_dir, 'data', 'line_lists', 'lines.txt' )

ldb = line_database.LineDatabase( linedb_fp )

ion_tuples = ldb.parse_subset_to_ions( species_list )

In [16]:
# Get names used by the code
density_names = []
den_name_dict = {}
for i, ( atom, ion) in enumerate( ion_tuples ):
    
    den_name = '{}_p{}_density'.format( atom, ion-1 )
    
    density_names.append( den_name )
        
    den_name_dict[species_list[i]] = den_name

## Center, rotate, etc

In [17]:
# Retrieve the center
try:
    c = []
    for coord_name in [ 'Xc', 'Yc', 'Zc' ]:
        c.append(
            hd.get_mt_data(
                coord_name,
                mt_halo_id = mt_halo_id,
                snums = [ snum ],
            )[0]
        )
    c = np.array( c ) * ds.quan( 1, 'kpccm/h' ) # Center in co-moving kpc/h
except NameError:
    _, c = ds.find_max( ('gas', 'density') )

c = c.to( 'kpc' ).value

yt : [INFO     ] 2020-09-16 19:44:13,380 Max Value is 1.39394e-20 at 29376.1624294746179658 30966.6151484254696697 32490.6046232758017140


In [18]:
thetay, thetaz, scom, vscom, gindices, sindices = extractDiskFromSnapdict(
    star_snap,
    gas_snap,
    25,
    scom = c,
)

In [19]:
rot_gas_snap = offsetRotateSnapshot(
    gas_snap,
    scom,
    vscom,
    thetay,
    thetaz,
    orient_stars = 1,
)

In [20]:
fiducial_masses = copy.deepcopy( rot_gas_snap['Masses'] )
fiducial_density = gas_snap['Density'] * 1e10 * unyt.msun / unyt.kpc**3.

/work/03057/zhafen/stampede2/miniconda3/lib/python3.7/site-packages/unyt/array.py:1693: RuntimeWarning: overflow encountered in multiply
  inp0.view(np.ndarray), inp1.view(np.ndarray), out=out_func, **kwargs


In [21]:
min_dens = {
    'C II': 13.,
    'Si II': 12,
    'Si III': 12,
    'Si IV': 12.5,
    'C IV': 13.,
    'O VI': 12,
}
max_dens = {
    'C II': 14.5,
    'Si II': 14.7,
    'Si III': 13.5,
    'Si IV': 14.5,
    'C IV': 14.5,
    'O VI': 15,
}

In [22]:
abundances = {}
for i, species in enumerate( species_list ):
    
    if use_stored_projections:
        print( 'Using stored {} densities'.format( species ) )
        # Unused abundances
        abundances[species] = np.ones( fiducial_density.shape )
        continue

    print( 'Computing {} densities'.format( species ) )
    
    abundances[species] = ds.all_data()[density_names[i]] / fiducial_density

Computing Si II densities
Computing C IV densities
Computing O VI densities


In [23]:
for i, species in enumerate( species_list ):

    print( 'Projecting {} densities'.format( species ) )

    abundance = abundances[species]

    rot_gas_snap['Masses'] = abundance * fiducial_masses

    my_studio = gas_studio.GasStudio(
        sim_dir,
        snum,
        fm.get_stained_glass_dir( **sim_kwargs ),
        frame_half_width = sidelength/2.,
        frame_depth = sightline_len,
        snapdict = rot_gas_snap,
        overwrite = not use_stored_projections,
        single_image = 'Density',
        use_colorbar = True,
        conv_fac = ( 1e10 * unyt.Msun / unyt.kpc**2. ).to( 'mp/cm**2').value,
        cmap = 'magma',
        min_den = 12,
        max_den = 18,
        scale_bar_length = 10.,
        intermediate_file_name = density_names[i],
        cbar_label = species_list[i] + r' column density (cm$^{-2}$)',
        theta = 90,
    )

    start_time = time.time()

    fig = plt.figure( figsize=(5, 5), facecolor='w' )
    ax = plt.gca()

    my_studio.render(
        ax,
        [ 'columnDensityMap', density_names[i] ],
        assert_cached = use_stored_projections,
    )

    print( 'Image generation took {:.3g} seconds'.format( time.time() - start_time ) )

    fig

Projecting Si II densities
extra kwargs:
 []
Drawing /scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_res7100/mhdcv/output:600 to:/scratch/03057/zhafen/stained_glass_data/cr_suite/m12i_res7100/mhdcv/
extracting cube
-done
Using provided smoothing lengths
------------------------------------------
------------------------------------------
log10 minmax(columnDensityMap) -inf 20.04486
log10 minmax(massWeightedQuantityMap) -inf -inf
min_rho =  12
max_rho =  18
Image range (rho):  -inf 20.04486
Image range (8bit):  0.0 255.0


/home1/03057/zhafen/repos/FIRE_studio/firestudio/studios/gas_studio.py:340: RuntimeWarning: divide by zero encountered in log10
  columnDensityMap = np.log10(columnDensityMap)
/home1/03057/zhafen/repos/FIRE_studio/firestudio/studios/gas_studio.py:348: RuntimeWarning: divide by zero encountered in log10
  massWeightedQuantityMap = np.log10(massWeightedQuantityMap)


Image saved at /scratch/03057/zhafen/stained_glass_data/cr_suite/m12i_res7100/mhdcv/Plots/GasTwoColour/Si_p1_density_600_600kpc.png
Image generation took 171 seconds
Projecting C IV densities
extra kwargs:
 []
Drawing /scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_res7100/mhdcv/output:600 to:/scratch/03057/zhafen/stained_glass_data/cr_suite/m12i_res7100/mhdcv/
extracting cube
-done
Using provided smoothing lengths
------------------------------------------
------------------------------------------
log10 minmax(columnDensityMap) 10.930087 17.053972
log10 minmax(massWeightedQuantityMap) 4.1080027 4.1080027
min_rho =  12
max_rho =  18
Image range (rho):  10.930087 17.053972
Image range (8bit):  0.0 214.79382
Image saved at /scratch/03057/zhafen/stained_glass_data/cr_suite/m12i_res7100/mhdcv/Plots/GasTwoColour/C_p3_density_600_600kpc.png
Image generation took 169 seconds
Projecting O VI densities
extra kwargs:
 []
Drawing /scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_res7100/